# Inscopix python API - imaging processing
### This notebook contains the procedure from preprocessing to ROI detection by CNMFe

### Import packages 
the inscopix python API is located in the same folder of data processing software. The isx.cnmfe is used for ROI detection.

In [1]:
import sys
sys.path.append("/Program Files/Inscopix/Data Processing/")
import os
import isx
import isx.cnmfe

from datetime import datetime
start_time = datetime.now()
print('Start time: {}'.format(start_time))

Start time: 2020-09-08 23:13:38.076111


### File path and recording names
Deinfe path and the recording names of our recordings.<br>

In [2]:
path_to_file = 'F:/inscopix_nVista3/'
data_dir = os.path.join(path_to_file, 'BES0225200908') # Only change here
file_names_in_path = []
file_names_in_path = sorted(set(os.listdir(data_dir)))

# rec_name = '2020-09-02-15-58-16_video'
rec_name = file_names_in_path[1].split('.')[0]#.split('-')[0]
# rec_name = file_names_in_path[8][:-8] # This should be the first/sec raw file in the list with the time stamp
series_rec_names = {
        'day_0':
        [
            rec_name,
        ]
}
print(data_dir)
print(series_rec_names)

F:/inscopix_nVista3/BES0225200908
{'day_0': ['2020-09-08-15-09-22_video']}


### Imaging Preprocessing, Sptail filter, Motion correction, dF/F, PCA-ICA, Event detection
Note the following cell will perfrom the whole procedure from preprocessing to event detection, it's written in a huge for loop, so keep the identation before all codes.<br>
There will be outputs of file names for each completed steps, so you can check the working progress.

In [3]:
# Process each series all the way to event detection.
output_dir = os.path.join(data_dir, 'processed_crop')
os.makedirs(output_dir, exist_ok=True)

proc_movie_files = []
proc_cs_files = []
for series_name, rec_names in series_rec_names.items():
    
    # Generate the recording file paths.
#     rec_files = [os.path.join(data_dir, name + '.xml') for name in rec_names]
    rec_files = [os.path.join(data_dir, name + '.isxd') for name in rec_names] ## This line is for new recording from nVista3
    print(rec_files) # for progress feedback, not necessary
    
    # Preprocess the recordings by spatially downsampling by a factor of 4.
    pp_files = isx.make_output_file_paths(rec_files, output_dir, 'PP')
    isx.preprocess(rec_files, pp_files, spatial_downsample_factor=1, crop_rect=[8,18,199,280])
    print(pp_files)
    
    # Perform spatial bandpass filtering with defaults.
    bp_files = isx.make_output_file_paths(pp_files, output_dir, 'BP')
    isx.spatial_filter(pp_files, bp_files, low_cutoff=0.005, high_cutoff=0.500)
    print(bp_files)
    
    # Motion correct the movies using the mean projection as a reference frame.
    mean_proj_file = os.path.join(output_dir, '{}-mean_image.isxd'.format(series_name))
    isx.project_movie(bp_files, mean_proj_file, stat_type='mean')
    mc_files = isx.make_output_file_paths(bp_files, output_dir, 'MC')
    translation_files = isx.make_output_file_paths(mc_files, output_dir, 'translations', 'csv')
    crop_rect_file = os.path.join(output_dir, '{}-crop_rect.csv'.format(series_name))
    isx.motion_correct(
         bp_files, mc_files, max_translation=20, reference_file_name=mean_proj_file,
         low_bandpass_cutoff=None, high_bandpass_cutoff=None,
         output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)
    print(mc_files)
    
#     # Run DF/F on the motion corrected movies.
#     dff_files = isx.make_output_file_paths(mc_files, output_dir, 'DFF')
#     isx.dff(mc_files, dff_files, f0_type='mean')
#     print(dff_files)
    
#     # Run PCA-ICA on the DF/F movies.
#     # Note that you will have to manually determine the number of cells, which we
#     # determined here as 180.
#     # Increase the block_size to increase speed at the expense of more memory usage.
#     ic_files = isx.make_output_file_paths(dff_files, output_dir, 'PCA_ICA')
#     num_cells = 180
#     isx.pca_ica(dff_files, ic_files, num_cells, int(1.15 * num_cells), block_size=1000)
#     #print(ic_files)

#     # Run event detection on the PCA-ICA cell sets.
#     event_files = isx.make_output_file_paths(ic_files, output_dir, 'ED')
#     isx.event_detection(ic_files, event_files, threshold=5)
#     print(event_files)

#     # Automatically accept and reject cells based on their cell metrics
#     # Only accept cells that have a nonzero event rate, an SNR greater
#     # than 3, and only one connected component after thresholding
#     auto_ar_filters = [('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)]
#     isx.auto_accept_reject(ic_files, event_files, filters=auto_ar_filters)

#     # Store the processed movies and cell sets for longitudinal registration.
#     proc_movie_files += dff_files
#     proc_cs_files += ic_files
#     print(proc_movie_files)

['F:/inscopix_nVista3/BES0225200908\\2020-09-08-15-09-22_video.isxd']
['F:/inscopix_nVista3/BES0225200908\\processed_crop\\2020-09-08-15-09-22_video-PP.isxd']
['F:/inscopix_nVista3/BES0225200908\\processed_crop\\2020-09-08-15-09-22_video-PP-BP.isxd']
['F:/inscopix_nVista3/BES0225200908\\processed_crop\\2020-09-08-15-09-22_video-PP-BP-MC.isxd']


In [4]:
end_time = datetime.now()
print('Duration from starting to MC: {}'.format(end_time - start_time))

Duration from starting to MC: 0:03:35.553330


You should see some outputs from each step of precessing above here.


If you have the MC.isxd file already, you can continue with following code.

'F:/inscopix_new/DSC7674200225\\processed\\recording_20200225_153103-PP-BP-MC.isxd'

### ROI detection by CNMFe
Now, we finished all preprocessing from the inscopix, and can move forward for Ca<sup>2</sup> ROI detection by CNMFe. First we would need to define our parameters.

In [5]:
# To start with CNMFe, we can try to set up a cluster on our computer so 
# it can process much faster. Go to anaconda promt, activate isxenv, then run
# ipcluster start -n 16

# Or less amount
# Then use a different terminal to start the jupyter notebook.
# Check again if it's possible to have this at the beginning of the notebook

In [6]:
## Test with different setting
# Define the motion correct filename for further process.
mov_file = str(mc_files) # This should have the full path to the motion corrected file.

# Create lists of CNMFe output files (cell traces and events)
cellset_list = []
cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_13'+ '.isxd'))
cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_17'+ '.isxd'))
cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_21'+ '.isxd'))
for cellset in cellset_list:
    print(cellset)

events_list = []
events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_13-ED'+ '.isxd'))
events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_17-ED'+ '.isxd'))
events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_21-ED'+ '.isxd'))
for events in events_list:
    print(events)

F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC-CNMFE_13.isxd
F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC-CNMFE_17.isxd
F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC-CNMFE_21.isxd
F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC-CNMFE_13-ED.isxd
F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC-CNMFE_17-ED.isxd
F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC-CNMFE_21-ED.isxd


In [7]:
start_time_2 = datetime.now()
print('Start time 2: {}'.format(start_time_2))

Start time 2: 2020-09-08 23:17:13.654922


In [8]:
# CNMFe parameter setting
# Define cell size, with the Ca2+ imaging from the inscopix (w/o crop VoF), 
# a 4x spatial downsampling video usually has a cell size around 15-20 pixel.
# Belows are some parameters that I tried that can give us nice yield of cells (~200-300 cells)

# Most important is the cell size here, change this one.
cell_size_px = 17       # commonly use: 13, 17, 21

px_set = [13,17,21]     # Use for automatic naming
gSig=(cell_size_px-1)/4 # gSig
gSiz=cell_size_px       # gSiz = gSig*4+1 (from CNMFe manual)
num_processes=1         # Check again if we can do multi-processor by changing the ipython kernel
K=None
rf=[40,40]
stride=20
min_pnr=10              # Try 20, 30 for the most activative neurons
min_corr=0.8            # Try 0.7 corr
event_threshold=0.3     # Used 0.1 before, too noisy

In [9]:
## Run CNMFe
# Automatic name the output file with corresponding cell size

isx.cnmfe.run_cnmfe(mc_files, 
                    cellset_list[px_set.index(cell_size_px)], 
                    events_list[px_set.index(cell_size_px)],
                    num_processes=num_processes, 
                    K=K, rf=rf, stride=stride,
                    gSig=gSig, gSiz=gSiz, 
                    min_pnr=min_pnr, min_corr=min_corr, 
                    event_threshold=event_threshold,
                    overwrite_tiff=True,
                    output_dir=output_dir)
print('Finished CNMFe '+cellset_list[px_set.index(cell_size_px)])

Exporting .isxd to tiff file(s)...
Wrote .tiff file to: F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC.tiff
Finished CNMFe F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP-MC-CNMFE_17.isxd


In [10]:
## Remove unnecessary files to sace storage space
os.remove(pp_files[0])
print('Removed '+str(pp_files[0]))
os.remove(bp_files[0])
print('Removed '+str(bp_files[0]))

Removed F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP.isxd
Removed F:/inscopix_nVista3/BES0225200908\processed_crop\2020-09-08-15-09-22_video-PP-BP.isxd


In [11]:
end_time = datetime.now()
print('End time: {}'.format(end_time))
print('Duration for CNMFe: {}'.format(end_time - start_time_2))
print('Duration for whole process: {}'.format(end_time - start_time))

End time: 2020-09-09 00:34:53.353911
Duration for CNMFe: 1:17:39.698989
Duration for whole process: 1:21:15.277800


Below is the original code for PCA and ICA, keep it here in case I need them sometime.

### Longitudinal registration
Now, we finished all individual file processings, following is the longitudinal registration from different days.
Following cells will also perform event detection and export individual cell maps, csv file of all Ca<sup>2+</sup> traces, registered movies (TIFF) 

In [12]:
# # Perform longitudinal registration on the processed movies and cell sets
# # to align the two days of data.
# lr_cs_files = isx.make_output_file_paths(proc_cs_files, output_dir, 'LR')
# lr_movie_files = isx.make_output_file_paths(proc_movie_files, output_dir, 'LR')
# lr_csv_file = os.path.join(output_dir, 'LR.csv')
# isx.longitudinal_registration(
#     proc_cs_files, lr_cs_files, input_movie_files=proc_movie_files,
#     output_movie_files=lr_movie_files, csv_file=lr_csv_file, accepted_cells_only=True)

In [13]:
# # Then run event detection and automatically classify the cells based on their
# # cell metrics.
# lr_event_files = isx.make_output_file_paths(lr_cs_files, output_dir, 'ED')
# isx.event_detection(lr_cs_files, lr_event_files, threshold=5)
# auto_ar_filters = [('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)]
# isx.auto_accept_reject(lr_cs_files, lr_event_files, filters=auto_ar_filters)

In [14]:
# # Finally, export the registered movies, cell sets, and event sets to non-native
# # formats (TIFF and CSV).
# tiff_movie_file = os.path.join(output_dir, 'DFF-LR.tif')
# isx.export_movie_to_tiff(lr_movie_files, tiff_movie_file, write_invalid_frames=True)
# tiff_image_file = os.path.join(output_dir, 'DFF-PCA_ICA-LR.tif')
# csv_trace_file = os.path.join(output_dir, 'DFF-PCA_ICA-LR.csv')
# isx.export_cell_set_to_csv_tiff(lr_cs_files, csv_trace_file, tiff_image_file, time_ref='start')
# csv_event_file = os.path.join(output_dir, 'DFF-PCA_ICA-LR-ED.csv')
# isx.export_event_set_to_csv(lr_event_files, csv_event_file, time_ref='start')